<a href="https://colab.research.google.com/github/ChaitaliV/Objective-criterias-to-quantify-the-accuracy-of-explanation/blob/main/Metrices/Stability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stability measures how well the explanation model is able to capture the behavior of the black box model under different conditions or perturbations. A higher stability indicates that the explanation model is more robust and reliable.



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install shap
!pip install lime
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
import pandas as pd
import numpy as np
import shap
from transformers import BertTokenizer
import scipy as sp
import torch
import lime
import tensorflow as tf
from lime.lime_text import LimeTextExplainer
import transformers
import torch.nn as nn
import random

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
model=torch.load(r'/content/drive/MyDrive/Saved Models/MuRIL.pt',map_location=torch.device('cuda'))
tokenizer = BertTokenizer.from_pretrained('google/muril-base-cased', do_lower_case=True)  

In [6]:
df = pd.read_csv('/content/drive/MyDrive/test.csv')
df.head()

,hindi_text,label
0,मैं न्यूयॉर्क में अपने दादा-दादी के साथ पैदा ह...,0
1,मैं किशोरावस्था से ही गंभीर अवसाद के दौर से गु...,1
2,मुझे शौक के तौर पर खाना बनाना पसंद है,0
3,"मैं इन दिनों बहुत कम महसूस कर रहा हूं, ऐसा महस...",1
4,हाल ही में ब्रिटेन की महारानी एलिजाबेथ का निधन...,2


In [7]:
test_data = df['hindi_text'][:30].tolist()
test_label = df['label'][:30].tolist()

In [8]:
# Define a function to preprocess the text data for the MuRIL model

def adapter(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      #m = model(**inputs)
      #print(model(**inputs).logits)
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()
      # predictions = tf.nn.softmax(logits)
      # prediction_label = class_name[np.argmax(predictions[0])]
  
  return scores

In [9]:
## define LIME explainer
class_name = ['Introduction','Depression','Grey Area']
explainer = LimeTextExplainer(class_names = class_name)

In [10]:
def lime_scores(lime_explanations):
  """
  extracts score from explanation tuples
  """
  lime_scores = []
  for tuple in lime_explanations:
    feature, score = tuple
    lime_scores.append(score)
  
  return lime_scores

def add_hindi_noise(text):
    """
    Adds 0.01 noise to each character in a Hindi text.
    """
    noise_text = ''
    for char in text:
        if char != ' ':
            unicode_val = ord(char)
            unicode_val += random.randint(-1, 1)
            noise_char = chr(unicode_val)
            noise_text += noise_char
        else:
            noise_text += ' '
    return noise_text

In [11]:
def calculate_lime_stability(texts, explainer, model, num_samples=15, threshold=0.1):
    num_texts = len(texts)
    num_features = 10
    expl1 = explainer.explain_instance(texts,adapter)
    exp1_as_list = lime_scores(expl1.as_list())
    score = []
    for j in range(num_samples):
        # Add noise to the input text
        noisy_text = add_hindi_noise(texts)
        expl2 = explainer.explain_instance(noisy_text,adapter)
        exp2_as_list = lime_scores(expl2.as_list())
        diff = np.abs(np.array(exp1_as_list) - np.array(exp2_as_list))
        diff_mean = np.mean(diff)
        score.append(diff_mean)
    score = np.array(score)
    lime_stability.append(np.sum(score < threshold) / float(len(score)))
    

In [19]:
lime_stability = []

In [20]:
calculate_lime_stability(test_data[3], explainer, model)

OutOfMemoryError: ignored